In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import mutual_info_classif
import datetime
import pickle
import scipy.sparse as ss
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
import os
from sklearn import linear_model, svm, neural_network, ensemble


import IPython.display as ipd
import copy
import random
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold,KFold

import torch.nn as nn
import math
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold,KFold

from sklearn.preprocessing import StandardScaler
from tqdm import tqdm_notebook as tqdm
import os

# Read

In [2]:
logging.info('start read')
df_master_records = pickle.load(open('../data_sortout/df_master_records.pickle', 'rb'))
se_id_install_list = pickle.load(open('../data_sortout/se_id_install_list.pickle', 'rb'))
df_install_behave = pickle.load(open('../data_sortout/df_install_behave_no_date.pickle', 'rb'))
df_behave_time = pickle.load(open('../data_sortout/df_time_cut.pickle', 'rb'))
se_userlog_cross = pickle.load(open('../data_sortout/se_userlog_cross_id.pickle', 'rb'))
logging.info('finish read')

2020-09-17 20:59:31,229 - INFO - start read
2020-09-17 20:59:58,386 - INFO - finish read


In [3]:
VAR_SAVE = 'var/baseline_'

# Feature

## user attribute

In [4]:
def get_master_user_discrete(df_master_records):
    
    df_master_records['qcut_amount_bin'] = pd.qcut(df_master_records['amount_bin'], 5)
    df_master_records['new_client'] = df_master_records['loan_sequence'] == 1
    df_master_records['qcut_age'] = pd.qcut(df_master_records['age'], 5, duplicates='drop')
    
    df_master_records['qcut_min_income'] = pd.qcut(df_master_records['min_income'], 6, duplicates='drop')
    df_master_records['qcut_max_income'] = pd.qcut(df_master_records['max_income'].apply(int), 6, duplicates='drop')
    
#     df_master_records['qcut_loan_sequence'] = pd.qcut(df_master_records['loan_sequence'], 6, duplicates='drop')

    pne_hot_cols = [ 'months', 'gender', 'educationid', 'marriagestatusid', 'income', 
                    'qcut_amount_bin', 'qcut_age', 'qcut_min_income', 'qcut_max_income']
    
    return  pd.get_dummies(df_master_records[pne_hot_cols], columns = pne_hot_cols)


## app

In [5]:
class MutualInfoSelection():
    
    def __init__(self, topN):
        self.topN = topN
    
    def get_select_vector(self, pkgs):
        """
        """
        v = np.zeros(len(self.mp_select_feature_order))
        for pkg in pkgs:
            if pkg in self.mp_select_feature_order:
                v[self.mp_select_feature_order[pkg]] = 1
        return v

    def fit(self, train_se_id_pkg_list, train_y):
        rows = []
        cols = []
        data = []
        for i in range(train_se_id_pkg_list.shape[0]):
            pkg_list = train_se_id_pkg_list.iloc[i]
            cols.extend(pkg_list)
            rows.extend([i] * len(pkg_list))
            data.extend([1] * len(pkg_list))
        train_sparse = ss.coo_matrix((data,(rows,cols)))

        mutual_weight = mutual_info_classif(train_sparse, train_y)
        select_feature = np.argsort(mutual_weight)[-self.topN:]
        self.mp_select_feature_order = {}
        self.mp_order_select_feature = {}

        for i, feature in enumerate(select_feature):
            self.mp_select_feature_order[feature] = i
            self.mp_order_select_feature[i] = feature
    
    def transform(self, se_id_pkg_list):
        se_app_feature = se_id_pkg_list.apply(self.get_select_vector)
        df_app_feature = pd.DataFrame(np.array(list(se_app_feature)))
        df_app_feature.index = se_app_feature.index

        return df_app_feature

# Train

## 数据划分

In [5]:
split_date = datetime.datetime(2019, 8, 31)
end_date = datetime.datetime(2019, 9, 30)

df_master_records = df_master_records.dropna(axis=0, how='any')
df_train_master = df_master_records.query('loan_date <= @split_date')
df_test_master = df_master_records.query('loan_date > @split_date & loan_date <= @end_date')
all_train_id = list(df_train_master.index)
all_test_id = list(df_test_master.index)
logging.info('all_train_id len :%d, all_test_id: %d' % (len(all_train_id), len(all_test_id)))

2020-09-17 20:59:58,723 - INFO - Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2020-09-17 20:59:58,724 - INFO - NumExpr defaulting to 8 threads.
2020-09-17 20:59:58,820 - INFO - all_train_id len :326082, all_test_id: 75896


## 共用函数

In [6]:
def train(prefix, dataset_dict, model):
    
    train_x = dataset_dict['train_x']
    train_y = dataset_dict['train_y']
    test_x = dataset_dict['test_x']
    test_y = dataset_dict['test_y']
    train_new_client = dataset_dict['train_new_client'].values
    test_new_client = dataset_dict['test_new_client'].values
    
#     sc = StandardScaler()
#     sc.fit(train_x)
#     train_x = sc.transform(train_x)
#     test_x = sc.transform(test_x)
    
    model.fit(train_x, train_y)
    
    predict_train = model.predict_proba(train_x)
    predict_test = model.predict_proba(test_x)
    
    auc_train = roc_auc_score(train_y, predict_train[:, 1])
    auc_test = roc_auc_score(test_y, predict_test[:, 1])
    
    new_auc_train = roc_auc_score(train_y[train_new_client], predict_train[:, 1][train_new_client])
    new_auc_test = roc_auc_score(test_y[test_new_client], predict_test[:, 1][test_new_client])

    old_auc_train = roc_auc_score(train_y[~train_new_client], predict_train[:, 1][~train_new_client])
    old_auc_test = roc_auc_score(test_y[~test_new_client], predict_test[:, 1][~test_new_client])

    train_ret_dict = {
        prefix : auc_train,
        "new_%s" % prefix: new_auc_train,
        "old_%s" % prefix : old_auc_train
    }
    
    test_ret_dict = {
        prefix : auc_test,
        "new_%s" % prefix: new_auc_test,
        "old_%s" % prefix : old_auc_test
    }
    return train_ret_dict, test_ret_dict

def get_dataset_user(df_feature, target_name, save_name, load_from_file = True):
    data_path = VAR_SAVE + save_name + target_name + '.pickle'
    if load_from_file and (os.path.exists(data_path)):
        return pickle.load(open(data_path, 'rb'))
    
    select_id = df_feature.index
    select_train_id = list( set(select_id) & set(all_train_id) )
    select_test_id = list( set(select_id) & set(all_test_id) )
    select_df_train = df_feature.loc[select_train_id]
    select_df_test = df_feature.loc[select_test_id]
    select_train_y = df_master_records.loc[select_train_id][target_name]
    select_test_y = df_master_records.loc[select_test_id][target_name]
    
    ret = {
        'train_x' : np.array(select_df_train),
        'train_y' : np.array(select_train_y),
        'test_x' : np.array(select_df_test),
        'test_y' : np.array(select_test_y),
        'train_new_client' : df_master_records.loc[select_train_id]['loan_sequence'] == 1,
        'test_new_client' : df_master_records.loc[select_test_id]['loan_sequence'] == 1
    }, select_df_train, select_df_test
    
    pickle.dump(ret, open(data_path, 'wb'), protocol = 4)
    return ret

def get_dataset_app(se_id_pkg, target_name, save_name, load_from_file = True):
    
    data_path = VAR_SAVE + save_name + target_name + '.pickle'
    if(load_from_file and os.path.exists(data_path)):
        return pickle.load(open(data_path, 'rb'))

    select_id = se_id_pkg.index
    select_train_id = list( set(select_id) & set(all_train_id) )
    select_test_id = list( set(select_id) & set(all_test_id) )
    
    app_selection = MutualInfoSelection(3000)
    select_train_y = df_master_records.loc[select_train_id][target_name]
    
    app_selection.fit(se_id_pkg.loc[select_train_id], select_train_y.values)
    logging.info('finish fit app_selection :%s' % target_name)
    select_df_train = app_selection.transform(se_id_pkg.loc[select_train_id])
    select_df_test = app_selection.transform(se_id_pkg.loc[select_test_id])
    
    select_test_y = df_master_records.loc[select_test_id][target_name]
    
    ret = {
        'train_x' : np.array(select_df_train),
        'train_y' : np.array(select_train_y),
        'test_x' : np.array(select_df_test),
        'test_y' : np.array(select_test_y),
        'train_new_client' : df_master_records.loc[select_train_id]['loan_sequence'] == 1,
        'test_new_client' : df_master_records.loc[select_test_id]['loan_sequence'] == 1
    }, select_df_train, select_df_test
    
    pickle.dump(ret, open(data_path, 'wb'), protocol = 4)
    return ret

def get_dataset_userlog(se_id_pkg, target_name, save_name, load_from_file = True):
    data_path = VAR_SAVE + save_name + target_name + '.pickle'
    if(load_from_file and os.path.exists(data_path)):
        return pickle.load(open(data_path, 'rb'))

    select_id = se_id_pkg.index
    select_train_id = list( set(select_id) & set(all_train_id) )
    select_test_id = list( set(select_id) & set(all_test_id) )
    
    se_train = se_id_pkg.loc[select_train_id]
    se_test = se_id_pkg.loc[select_test_id]
    
    userlog_action = 137
    train_np = np.zeros((se_train.shape[0], userlog_action))
    test_np = np.zeros((se_test.shape[0], userlog_action))
    
    for i in tqdm(range(se_train.shape[0])):
        for item in se_train.iloc[i]:
            train_np[i][item-1] += 1
    
    
    for i in tqdm(range(se_test.shape[0])):
        for item in se_test.iloc[i]:
            test_np[i][item-1] += 1
    
    select_df_train = pd.DataFrame(train_np, index=se_train.index)
    select_df_test = pd.DataFrame(test_np, index=se_test.index)
    select_train_y = df_master_records.loc[select_train_id][target_name]
    select_test_y = df_master_records.loc[select_test_id][target_name]

    ret = {
        'train_x' : np.array(select_df_train),
        'train_y' : np.array(select_train_y),
        'test_x' : np.array(select_df_test),
        'test_y' : np.array(select_test_y),
        'train_new_client' : df_master_records.loc[select_train_id]['loan_sequence'] == 1,
        'test_new_client' : df_master_records.loc[select_test_id]['loan_sequence'] == 1
    }, select_df_train, select_df_test
    
    pickle.dump(ret, open(data_path, 'wb'), protocol = 4)
    return ret

def get_dataset_merge(df_feature, target_name):
    
    select_id = df_feature.index
    select_train_id = list( set(select_id) & set(all_train_id) )
    select_test_id = list( set(select_id) & set(all_test_id) )
    select_df_train = df_feature.loc[select_train_id]
    select_df_test = df_feature.loc[select_test_id]
    select_train_y = df_master_records.loc[select_train_id][target_name]
    select_test_y = df_master_records.loc[select_test_id][target_name]
    
    ret = {
        'train_x' : np.array(select_df_train),
        'train_y' : np.array(select_train_y),
        'test_x' : np.array(select_df_test),
        'test_y' : np.array(select_test_y),
        'train_new_client' : df_master_records.loc[select_train_id]['loan_sequence'] == 1,
        'test_new_client' : df_master_records.loc[select_test_id]['loan_sequence'] == 1
    }
    
    return ret


## user attribute

In [8]:
df_user_one_hot = get_master_user_discrete(df_master_records)
user_train_df_list, user_test_df_list = [], []

for i, model in enumerate([
    linear_model.LogisticRegression(), 
    svm.SVC(probability=True, max_iter = 100),
    neural_network.MLPClassifier(random_state=0, max_iter=300),
    ensemble.GradientBoostingClassifier(random_state=0),
]):
    print(model)
    
    train_ret_dict, test_ret_dict = {}, {}
    

    for j, target_name in enumerate(['1m30+', '2m30+', '3m30+', '4m30+']):
        user_dataset_dict, user_train_df, user_test_df = get_dataset_user(df_user_one_hot, 'target_%s' % target_name, 'user_info_')
        if i == 0:
            user_train_df_list.append(user_train_df)
            user_test_df_list.append(user_test_df)
        
        logging.info('start train user attribute:%s' % target_name)
        use_rets = train(target_name, user_dataset_dict, model)
        train_ret_dict.update(use_rets[0])
        test_ret_dict.update(use_rets[1])
    
    df_ret = pd.DataFrame([
        train_ret_dict, test_ret_dict
    ], index = ['train', 'test'])
    ipd.display(df_ret)

2020-09-15 13:51:27,273 - INFO - start train user attribute:1m30+


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


2020-09-15 13:51:31,602 - INFO - start train user attribute:2m30+
2020-09-15 13:51:35,948 - INFO - start train user attribute:3m30+
2020-09-15 13:51:40,680 - INFO - start train user attribute:4m30+


,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.625875,0.618872,0.630604,0.626575,0.628666,0.632849,0.610282,0.613838,0.605459,0.611324,0.610458,0.607270
test,0.596442,0.603697,0.626734,0.597912,0.612690,0.603555,0.584626,0.596219,0.587037,0.592436,0.593697,0.598568


2020-09-15 13:51:44,988 - INFO - start train user attribute:1m30+


...

,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.690940,0.671622,0.709116,0.672593,0.663464,0.687571,0.657897,0.647829,0.662453,0.656693,0.642248,0.663474
test,0.578484,0.578145,0.581842,0.592899,0.603298,0.603046,0.588560,0.592383,0.592671,0.595166,0.590465,0.605129


2020-09-15 14:32:52,938 - INFO - start train user attribute:1m30+


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


2020-09-15 14:33:58,922 - INFO - start train user attribute:2m30+
2020-09-15 14:35:04,572 - INFO - start train user attribute:3m30+
2020-09-15 14:36:09,905 - INFO - start train user attribute:4m30+


,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.638518,0.626485,0.644690,0.634032,0.633305,0.641076,0.617661,0.618238,0.613377,0.618672,0.614344,0.615477
test,0.601561,0.606003,0.626293,0.604467,0.617741,0.609384,0.590246,0.600103,0.592491,0.597673,0.597783,0.603107


## App list

In [9]:
applist_train_df_list, applist_test_df_list = [], []

for i, model in enumerate([
    linear_model.LogisticRegression(), 
    svm.SVC(probability=True, max_iter = 100),
    neural_network.MLPClassifier(random_state=0, max_iter=300),
    ensemble.GradientBoostingClassifier(random_state=0), 
]):
    print(model)
    train_ret_dict, test_ret_dict = {}, {}

    for j, target_name in enumerate(['1m30+', '2m30+', '3m30+', '4m30+']):
        logging.info('start train app list:%s' % target_name)
        app_list_dataset_dict, applist_train_df, applist_test_df = \
        get_dataset_app(se_id_install_list, 'target_%s' % target_name, 'app_list_')

        if i == 0:
            applist_train_df_list.append(applist_train_df)
            applist_test_df_list.append(applist_test_df)
            
        use_rets = train(target_name, app_list_dataset_dict, model)
        train_ret_dict.update(use_rets[0])
        test_ret_dict.update(use_rets[1])
    df_ret = pd.DataFrame([
        train_ret_dict, test_ret_dict
    ], index = ['train', 'test'])
    ipd.display(df_ret)

2020-09-15 14:37:15,335 - INFO - start train app list:1m30+


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


2020-09-15 15:08:27,910 - INFO - finish fit app_selection :target_1m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
2020-09-15 15:09:43,105 - INFO - start train app list:2m30+
2020-09-15 15:41:07,197 - INFO - finish fit app_selection :target_2m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (m

,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.795124,0.756913,0.826056,0.762485,0.718350,0.795502,0.758677,0.707241,0.794341,0.752685,0.699539,0.784836
test,0.624672,0.590205,0.630476,0.632811,0.612104,0.624475,0.630766,0.601225,0.628878,0.624073,0.595976,0.616754


2020-09-15 16:47:49,212 - INFO - start train app list:1m30+


...

,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.977760,0.977451,0.969665,0.982641,0.980343,0.983993,0.981999,0.982164,0.978482,0.978154,0.979823,0.972661
test,0.577342,0.557984,0.569117,0.613408,0.600338,0.607838,0.632747,0.608012,0.639399,0.639594,0.613674,0.641670


2020-09-15 17:46:21,479 - INFO - start train app list:1m30+


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


2020-09-15 18:10:31,529 - INFO - start train app list:2m30+
2020-09-15 18:38:39,973 - INFO - start train app list:3m30+
2020-09-15 19:07:17,146 - INFO - start train app list:4m30+


,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.677222,0.634601,0.722254,0.685252,0.650932,0.715057,0.680150,0.640452,0.703444,0.674307,0.634275,0.691556
test,0.612065,0.584718,0.591118,0.642356,0.622704,0.641459,0.629333,0.604197,0.624565,0.619844,0.595450,0.609771


## App install behave

In [10]:
appbehave_df_train, appbehave_df_test = [], []

for i, model in enumerate([
    linear_model.LogisticRegression(),
    svm.SVC(probability=True, max_iter = 100),
    neural_network.MLPClassifier(random_state=0, max_iter=300),
    ensemble.GradientBoostingClassifier(random_state=0),
]):
    print(model)
    train_ret_dict, test_ret_dict = {}, {}
    for j, target_name in enumerate(['1m30+', '2m30+', '3m30+', '4m30+']):
        logging.info('start train app behave:%s' % target_name)
        app_behave_dataset_dict, behave_df_train, behave_df_test = \
            get_dataset_app(df_install_behave['pkg_id'], 'target_%s' % target_name, 'app_behave_')

        if i == 0:
            appbehave_df_train.append(behave_df_train)
            appbehave_df_test.append(behave_df_test)
            
        use_rets = train(target_name, app_behave_dataset_dict, model)
        train_ret_dict.update(use_rets[0])
        test_ret_dict.update(use_rets[1])
    df_ret = pd.DataFrame([
        train_ret_dict, test_ret_dict
    ], index = ['train', 'test'])
    ipd.display(df_ret)

2020-09-15 19:37:38,760 - INFO - start train app behave:1m30+


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


2020-09-15 20:46:25,740 - INFO - finish fit app_selection :target_1m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
2020-09-15 20:47:43,176 - INFO - start train app behave:2m30+
2020-09-15 21:55:34,190 - INFO - finish fit app_selection :target_2m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations 

,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.746595,0.716207,0.788276,0.710741,0.671820,0.738240,0.706321,0.665432,0.733955,0.700920,0.658793,0.727018
test,0.586649,0.557882,0.614600,0.592539,0.570734,0.586963,0.584595,0.562763,0.578391,0.584154,0.563281,0.577076


2020-09-16 00:19:14,328 - INFO - start train app behave:1m30+


...

,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.967563,0.968410,0.957748,0.995129,0.996083,0.993663,0.993781,0.994940,0.992477,0.993694,0.995779,0.991014
test,0.548351,0.534258,0.565948,0.598249,0.587642,0.594681,0.615758,0.599314,0.616125,0.623532,0.597470,0.633484


2020-09-16 03:39:24,172 - INFO - start train app behave:1m30+


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


2020-09-16 04:10:06,121 - INFO - start train app behave:2m30+
2020-09-16 04:41:45,827 - INFO - start train app behave:3m30+
2020-09-16 05:13:32,509 - INFO - start train app behave:4m30+


,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.627226,0.589601,0.651348,0.634683,0.603308,0.651643,0.630418,0.596342,0.644108,0.627819,0.597356,0.634972
test,0.592185,0.548605,0.611608,0.594711,0.565668,0.590668,0.588176,0.562709,0.575235,0.585659,0.563029,0.569546


## userlog

In [11]:
userlog_df_train_list, userlog_df_test_list, userlog_dataset_dict_list = [], [], []
for i, model in enumerate([
    linear_model.LogisticRegression(), 
    svm.SVC(probability=True, max_iter = 100),
    neural_network.MLPClassifier(random_state=0, max_iter=300),
    ensemble.GradientBoostingClassifier(random_state=0),
]):
    print(model)
    
    train_ret_dict, test_ret_dict = {}, {}
    for j,target_name in enumerate(['1m30+', '2m30+', '3m30+', '4m30+']):
        logging.info('start train user log:%s' % target_name)
        userlog_dataset_dict, userlog_df_train, userlog_df_test = \
        get_dataset_userlog(se_userlog_cross, 'target_%s' % target_name, 'user_log_')

        if  i == 0:
            userlog_df_train_list.append(userlog_df_train)
            userlog_df_test_list.append(userlog_df_test)
    
        use_rets = train(target_name, userlog_dataset_dict, model)
        train_ret_dict.update(use_rets[0])
        test_ret_dict.update(use_rets[1])
    df_ret = pd.DataFrame([
        train_ret_dict, test_ret_dict
    ], index = ['train', 'test'])
    ipd.display(df_ret)

2020-09-16 05:45:12,584 - INFO - start train user log:1m30+


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
2020-09-16 05:47:41,695 - INFO - start train user log:2m30+


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
2020-09-16 05:50:09,795 - INFO - start train user log:3m30+


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
2020-09-16 05:52:40,478 - INFO - start train user log:4m30+


...

,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.901060,0.861956,0.951442,0.847345,0.801905,0.896954,0.822848,0.77205,0.859428,0.809958,0.763863,0.843430
test,0.559621,0.529833,0.505122,0.622999,0.606327,0.617535,0.622039,0.60529,0.619803,0.618987,0.609627,0.617266


2020-09-16 06:56:31,720 - INFO - start train user log:1m30+


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


2020-09-16 07:00:53,622 - INFO - start train user log:2m30+
2020-09-16 07:05:09,699 - INFO - start train user log:3m30+
2020-09-16 07:09:24,519 - INFO - start train user log:4m30+


,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.712533,0.670420,0.748762,0.715196,0.70528,0.745675,0.685342,0.678060,0.694815,0.668651,0.668979,0.665436
test,0.687336,0.652753,0.704740,0.702833,0.69805,0.708206,0.661784,0.665527,0.658889,0.642620,0.654038,0.634551


## merge (drop missing)

In [ ]:
def merge_df_list(user_df_list, applist_df_list, appbehave_df, userlog_list, drop = True):
    df_merge_list = []
    logging.info('start merge')
    for (df_user_attribute, df_applist, df_appbehave, df_userlog) in zip(user_df_list, applist_df_list, appbehave_df, userlog_list):
        df_merge = pd.concat([df_user_attribute, df_applist, df_appbehave, df_userlog], axis=1)
        if drop:
            df_merge = df_merge.dropna()
        else:
            df_merge = df_merge.fillna(0)
        df_merge_list.append(df_merge)
    logging.info('finish merge')
    return df_merge_list

In [ ]:
df_merge_train_list = merge_df_list(user_train_df_list, applist_train_df_list, appbehave_df_train, userlog_df_train_list)
df_merge_test_list = merge_df_list(user_test_df_list, applist_test_df_list, appbehave_df_test, userlog_df_test_list)

In [14]:
for i, model in enumerate([
    linear_model.LogisticRegression(), 
    svm.SVC(probability=True, max_iter = 100),
    neural_network.MLPClassifier(random_state=0, max_iter=300),
    ensemble.GradientBoostingClassifier(random_state=0),
]):
    print(model)
    train_ret_dict, test_ret_dict = {}, {}
    for j, target_name in enumerate(['1m30+', '2m30+', '3m30+', '4m30+']):
        df_merge = pd.concat([df_merge_train_list[j], df_merge_test_list[j]])
        merge_dataset = get_dataset_merge(df_merge, 'target_%s' % target_name)
        logging.info('start train merge attribute:%s' % target_name)
        merge_rets = train(target_name, merge_dataset, model)
        train_ret_dict.update(merge_rets[0])
        test_ret_dict.update(merge_rets[1])

    df_ret = pd.DataFrame([
        train_ret_dict, test_ret_dict
    ], index = ['train', 'test'])
    ipd.display(df_ret)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


2020-09-16 07:18:26,900 - INFO - start train merge attribute:1m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
2020-09-16 07:19:29,479 - INFO - start train merge attribute:2m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/s

,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.911924,0.888882,0.934762,0.869140,0.828170,0.897205,0.859146,0.811956,0.888970,0.856215,0.804395,0.885391
test,0.637375,0.602558,0.664680,0.685467,0.662471,0.690375,0.670715,0.649591,0.665974,0.669940,0.648682,0.662887


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=100, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


2020-09-16 07:23:09,980 - INFO - start train merge attribute:1m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
2020-09-16 07:39:08,523 - INFO - start train merge attribute:2m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
2020-09-16 07:55:35,885 - INFO - start train merge attribute:3m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
2020-09-16 08

...

2020-09-16 08:31:04,528 - INFO - start train merge attribute:1m30+
2020-09-16 08:35:22,359 - INFO - start train merge attribute:2m30+
2020-09-16 08:46:00,958 - INFO - start train merge attribute:3m30+
2020-09-16 08:59:28,221 - INFO - start train merge attribute:4m30+


,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.992900,0.994499,0.986414,0.999261,0.999756,0.998661,0.998939,0.999435,0.998288,0.999068,0.999694,0.998340
test,0.638879,0.614047,0.648293,0.680080,0.663298,0.695996,0.673979,0.649748,0.683797,0.682228,0.664569,0.681397


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


2020-09-16 09:15:42,950 - INFO - start train merge attribute:1m30+
2020-09-16 09:29:40,542 - INFO - start train merge attribute:2m30+
2020-09-16 09:44:18,154 - INFO - start train merge attribute:3m30+
2020-09-16 09:59:03,332 - INFO - start train merge attribute:4m30+


,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.752037,0.704672,0.823028,0.764735,0.732800,0.805373,0.746147,0.710043,0.772522,0.743735,0.708155,0.762506
test,0.694404,0.649703,0.710970,0.719814,0.701277,0.728671,0.687447,0.672858,0.680801,0.686749,0.675220,0.680211


## merge fill zeros

In [ ]:
# df_merge_train_list = merge_df_list(user_train_df_list, applist_train_df_list, appbehave_df_train, userlog_df_train_list, False)
# df_merge_test_list = merge_df_list(user_test_df_list, applist_test_df_list, appbehave_df_test, userlog_df_test_list,  False)

2020-09-16 10:13:46,458 - INFO - start merge


In [ ]:
# merge_dataset_list = []
# for i, model in enumerate([
#     linear_model.LogisticRegression(), 
#     svm.SVC(probability=True, max_iter = 100),
#     neural_network.MLPClassifier(random_state=0, max_iter=300),
#     ensemble.GradientBoostingClassifier(random_state=0),
# ]):
#     print(model)
#     train_ret_dict, test_ret_dict = {}, {}
#     for j, target_name in enumerate(['1m30+', '2m30+', '3m30+', '4m30+']):
#         df_merge = pd.concat([df_merge_train_list[j], df_merge_test_list[j]])
#         merge_dataset = get_dataset_merge(df_merge, 'target_%s' % target_name)
#         merge_dataset_list.append(merge_dataset)

#         logging.info('start train merge fill zeros attribute:%s' % target_name)
#         merge_rets = train(target_name, user_dataset_dict, model)
#         train_ret_dict.update(merge_rets[0])
#         test_ret_dict.update(merge_rets[1])

#     df_ret = pd.DataFrame([
#         train_ret_dict, test_ret_dict
#     ], index = ['train', 'test'])
#     ipd.display(df_ret)

In [7]:
def get_df_merge_dataset(target_name):
    logging.info('start loading %s' % target_name)
    _, df_train_user_info, df_test_user_info = pickle.load(open('var/baseline_user_info_%s.pickle' % target_name, 'rb'))
    _, df_train_app_list, df_test_app_list = pickle.load(open('var/baseline_app_list_%s.pickle' % target_name, 'rb'))
    _, df_train_app_behave, df_test_app_behave = pickle.load(open('var/baseline_app_behave_%s.pickle' % target_name, 'rb')) 
    _, df_train_user_log, df_test_user_log = pickle.load(open('var/baseline_user_log_%s.pickle' % target_name, 'rb')) 
    df_merge_train = pd.concat([df_train_user_info, df_train_app_list, df_train_app_behave, df_train_user_log], axis=1)
    df_merge_train = df_merge_train.fillna(0)
    df_merge_test = pd.concat([df_test_user_info, df_test_app_list, df_test_app_behave, df_test_user_log], axis=1)
    df_merge_test = df_merge_test.fillna(0)
 
    select_train_id = df_merge_train.index
    select_test_id = df_merge_test.index 
    select_train_y = df_master_records.loc[select_train_id][target_name]
    select_test_y = df_master_records.loc[select_test_id][target_name]
    
    ret = {
        'train_x' : np.array(df_merge_train),
        'train_y' : np.array(select_train_y),
        'test_x' : np.array(df_merge_test),
        'test_y' : np.array(select_test_y),
        'train_new_client' : df_master_records.loc[select_train_id]['loan_sequence'] == 1,
        'test_new_client' : df_master_records.loc[select_test_id]['loan_sequence'] == 1
    }

    logging.info('finish loading %s' % target_name)

    return ret

In [ ]:
for i, model_orgin in enumerate([
    linear_model.LogisticRegression(), 
    svm.SVC(probability=True, max_iter = 100),
    neural_network.MLPClassifier(random_state=0, max_iter=300),
    ensemble.GradientBoostingClassifier(random_state=0),
]):
    model = copy.deepcopy(model_orgin)
    print(model)
    train_ret_dict, test_ret_dict = {}, {}
    for j, target_name in enumerate(['1m30+', '2m30+', '3m30+', '4m30+']):
        merge_dataset_dict = get_df_merge_dataset('target_%s' % target_name)
        logging.info('start train merge fill zeros attribute:%s' % target_name)
        merge_rets = train(target_name, merge_dataset_dict, model)
        train_ret_dict.update(merge_rets[0])
        test_ret_dict.update(merge_rets[1])

    df_ret = pd.DataFrame([
        train_ret_dict, test_ret_dict
    ], index = ['train', 'test'])
    ipd.display(df_ret)

2020-09-16 23:03:11,477 - INFO - start loading target_1m30+


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


2020-09-16 23:05:56,859 - INFO - finish loading target_1m30+
2020-09-16 23:05:57,141 - INFO - start train merge fill zeros attribute:1m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
2020-09-16 23:09:03,738 - INFO - start loading target_2m30+
2020-09-16 23:13:33,458 - INFO - finish loading target_2m30+
2020-09-16 23:13:33,818 - INFO - start train merge fill zeros attribute:2m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: Conve

,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.649282,0.610092,0.627930,0.650687,0.635481,0.677163,0.640359,0.632646,0.643596,0.599035,0.603866,0.596196
test,0.639917,0.599173,0.631661,0.667272,0.658869,0.690338,0.641780,0.637250,0.645740,0.607867,0.616288,0.611415


2020-09-16 23:29:06,808 - INFO - start loading target_1m30+


...

2020-09-16 23:32:02,755 - INFO - finish loading target_1m30+
2020-09-16 23:32:03,149 - INFO - start train merge fill zeros attribute:1m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
2020-09-17 00:26:18,048 - INFO - start loading target_2m30+
2020-09-17 00:30:48,225 - INFO - finish loading target_2m30+
2020-09-17 00:30:48,625 - INFO - start train merge fill zeros attribute:2m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
2020-09-17 01:25:13,421 - INFO - start loading target_3m30+
2020-09-17 01:29:55,996 - INFO - finish loading target_3m30+
2020-09-17 01:29:

,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.501867,0.443054,0.299989,0.424183,0.395770,0.318497,0.451554,0.413589,0.360563,0.489084,0.462056,0.407093
test,0.425417,0.374139,0.300584,0.391074,0.360075,0.342503,0.423772,0.384508,0.385690,0.453559,0.413899,0.412410


2020-09-17 03:23:35,393 - INFO - start loading target_1m30+


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=300,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=0, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)


2020-09-17 03:27:10,666 - INFO - finish loading target_1m30+
2020-09-17 03:27:11,069 - INFO - start train merge fill zeros attribute:1m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
2020-09-17 08:31:40,102 - INFO - start loading target_2m30+


In [8]:
for i, model_orgin in enumerate([
    ensemble.GradientBoostingClassifier(random_state=0),
]):
    model = copy.deepcopy(model_orgin)
    print(model)
    train_ret_dict, test_ret_dict = {}, {}
    for j, target_name in enumerate(['1m30+', '2m30+', '3m30+', '4m30+']):
        merge_dataset_dict = get_df_merge_dataset('target_%s' % target_name)
        logging.info('start train merge fill zeros attribute:%s' % target_name)
        merge_rets = train(target_name, merge_dataset_dict, model)
        train_ret_dict.update(merge_rets[0])
        test_ret_dict.update(merge_rets[1])

    df_ret = pd.DataFrame([
        train_ret_dict, test_ret_dict
    ], index = ['train', 'test'])
    ipd.display(df_ret)

2020-09-17 08:33:40,929 - INFO - start loading target_1m30+


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


2020-09-17 08:35:52,158 - INFO - finish loading target_1m30+
2020-09-17 08:35:52,462 - INFO - start train merge fill zeros attribute:1m30+
2020-09-17 09:27:48,502 - INFO - start loading target_2m30+
2020-09-17 09:31:40,280 - INFO - finish loading target_2m30+
2020-09-17 09:31:40,688 - INFO - start train merge fill zeros attribute:2m30+
2020-09-17 10:22:39,173 - INFO - start loading target_3m30+
2020-09-17 10:26:55,550 - INFO - finish loading target_3m30+
2020-09-17 10:26:55,954 - INFO - start train merge fill zeros attribute:3m30+
2020-09-17 11:16:54,330 - INFO - start loading target_4m30+
2020-09-17 11:19:18,437 - INFO - finish loading target_4m30+
2020-09-17 11:19:18,826 - INFO - start train merge fill zeros attribute:4m30+


,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.723536,0.666592,0.752051,0.724728,0.691111,0.760201,0.701055,0.669679,0.717257,0.692881,0.666683,0.698521
test,0.685207,0.639755,0.697357,0.702941,0.683918,0.713975,0.672260,0.661397,0.671495,0.669371,0.662018,0.668073


In [15]:
import torch
import torch.nn.functional as F
import torch.utils.data as Data
import torch.nn as nn
from transformers import *

class GeLU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1. + torch.tanh(x * 0.7978845608 * (1. + 0.044715 * x * x)))

class DNN(nn.Module):
    def __init__(self, in_feature):
        super().__init__()
        hidden = 128
        self.dense = nn.Sequential(
            nn.Linear(in_feature, hidden),
            GeLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden, 2)
        )
        self.dense.apply(self._init_weights)
        
    def _init_weights(self, module):
        """ Initialize the weights """
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)

    def forward(self, x):
        return self.dense(x)

class DNNTrain():
    def __init__(self):
        self.model = None
        
    def fit(self, X, y):
        self.model = DNN(X.shape[1]).cuda()
        self.model.train()

        torch_dataset = Data.TensorDataset(torch.tensor(X.astype('float32')), torch.tensor(y.astype('int')))
        data_loader = Data.DataLoader(
            dataset=torch_dataset,      
            batch_size=1024,      
            shuffle=True,
            num_workers = 0,
        )

        optimizer = AdamW(self.model.parameters(), lr = 0.01, weight_decay = 0.01)
 
        loss_func = nn.CrossEntropyLoss()

        for epoch in range(12):
            self.model.train()
            for i, (batch_x, batch_y) in enumerate(tqdm(data_loader)):
                batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
                pre = self.model(batch_x)    
                loss = loss_func(pre, batch_y.long())
                #backward
                S = time.time()
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm = 2)
                optimizer.step()
                
    def predict_proba(self, X):
        self.model.eval()
        torch_dataset = Data.TensorDataset(torch.tensor(X.astype('float32')))
        data_loader = Data.DataLoader(
            dataset=torch_dataset,      
            batch_size=1024,      
            shuffle=True,
            num_workers = 0,
        )
        pre_list = []
        for i, (batch_x,) in enumerate(tqdm(data_loader)):
            pre = self.model(batch_x.cuda())    
            pre_list.append(pre.cpu().detach().numpy())
        return np.concatenate(pre_list,  axis = 0)

In [8]:
train_ret_dict, test_ret_dict = {}, {}
for j, target_name in enumerate(['1m30+', '2m30+', '3m30+', '4m30+']):
    merge_dataset_dict = get_df_merge_dataset('target_%s' % target_name)
    logging.info('start train merge fill zeros attribute:%s' % target_name)
    merge_rets = train(target_name, merge_dataset_dict, neural_network.MLPClassifier(random_state=0, max_iter=100))
    print(merge_rets)
    train_ret_dict.update(merge_rets[0])
    test_ret_dict.update(merge_rets[1])

df_ret = pd.DataFrame([
    train_ret_dict, test_ret_dict
], index = ['train', 'test'])
ipd.display(df_ret)

2020-09-17 20:59:59,026 - INFO - start loading target_1m30+
2020-09-17 21:03:39,444 - INFO - finish loading target_1m30+
2020-09-17 21:03:39,793 - INFO - start train merge fill zeros attribute:1m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
2020-09-18 00:33:15,522 - INFO - start loading target_2m30+


({'1m30+': 0.9681381926324119, 'new_1m30+': 0.9661895860202934, 'old_1m30+': 0.9667543195498499}, {'1m30+': 0.6139951822743608, 'new_1m30+': 0.5984622938978599, 'old_1m30+': 0.6646996225146123})


2020-09-18 00:35:42,789 - INFO - finish loading target_2m30+
2020-09-18 00:35:43,133 - INFO - start train merge fill zeros attribute:2m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
2020-09-18 03:57:23,329 - INFO - start loading target_3m30+


({'2m30+': 0.952655145976405, 'new_2m30+': 0.9533724411242775, 'old_2m30+': 0.9523938761926372}, {'2m30+': 0.6583299550320685, 'new_2m30+': 0.6424596104267862, 'old_2m30+': 0.6754701047573929})


2020-09-18 04:01:14,815 - INFO - finish loading target_3m30+
2020-09-18 04:01:15,284 - INFO - start train merge fill zeros attribute:3m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
2020-09-18 07:02:24,217 - INFO - start loading target_4m30+


({'3m30+': 0.9483469705596369, 'new_3m30+': 0.9483583472457046, 'old_3m30+': 0.9474853229647335}, {'3m30+': 0.6502197498254194, 'new_3m30+': 0.633652861195702, 'old_3m30+': 0.6753175402839878})


2020-09-18 07:06:49,856 - INFO - finish loading target_4m30+
2020-09-18 07:06:50,309 - INFO - start train merge fill zeros attribute:4m30+
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


({'4m30+': 0.9457503700976314, 'new_4m30+': 0.9449680612526397, 'old_4m30+': 0.9446924681860115}, {'4m30+': 0.6434467779975812, 'new_4m30+': 0.6212707017692016, 'old_4m30+': 0.6686445813318918})


,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.968138,0.966190,0.966754,0.952655,0.953372,0.952394,0.948347,0.948358,0.947485,0.945750,0.944968,0.944692
test,0.613995,0.598462,0.664700,0.658330,0.642460,0.675470,0.650220,0.633653,0.675318,0.643447,0.621271,0.668645


## App concat

In [8]:
def get_app_merge_dataset(target_name):
    logging.info('start loading %s' % target_name)
    _, df_train_app_list, df_test_app_list = pickle.load(open('var/baseline_app_list_%s.pickle' % target_name, 'rb'))
    _, df_train_app_behave, df_test_app_behave = pickle.load(open('var/baseline_app_behave_%s.pickle' % target_name, 'rb')) 
    df_merge_train = pd.concat([df_train_app_list, df_train_app_behave], axis=1)
    df_merge_train = df_merge_train.fillna(0)
    df_merge_test = pd.concat([df_test_app_list, df_test_app_behave], axis=1)
    df_merge_test = df_merge_test.fillna(0)
 
    select_train_id = df_merge_train.index
    select_test_id = df_merge_test.index 
    select_train_y = df_master_records.loc[select_train_id][target_name]
    select_test_y = df_master_records.loc[select_test_id][target_name]
    
    ret = {
        'train_x' : np.array(df_merge_train),
        'train_y' : np.array(select_train_y),
        'test_x' : np.array(df_merge_test),
        'test_y' : np.array(select_test_y),
        'train_new_client' : df_master_records.loc[select_train_id]['loan_sequence'] == 1,
        'test_new_client' : df_master_records.loc[select_test_id]['loan_sequence'] == 1
    }

    logging.info('finish loading %s' % target_name)

    return ret

In [9]:
for i, model_orgin in enumerate([
#     linear_model.LogisticRegression(), 
#     svm.SVC(probability=True, max_iter = 100),
    neural_network.MLPClassifier(random_state=0, max_iter=300),
#     ensemble.GradientBoostingClassifier(random_state=0),
]):
    model = copy.deepcopy(model_orgin)
    print(model)
    
    train_ret_dict, test_ret_dict = {}, {}
    for j, target_name in enumerate(['1m30+', '2m30+', '3m30+', '4m30+']):
        merge_dataset_dict = get_app_merge_dataset('target_%s' % target_name)
        logging.info('start train merge fill zeros attribute:%s' % target_name)
        merge_rets = train(target_name, merge_dataset_dict, model)
        train_ret_dict.update(merge_rets[0])
        test_ret_dict.update(merge_rets[1])

    df_ret = pd.DataFrame([
        train_ret_dict, test_ret_dict
    ], index = ['train', 'test'])
    ipd.display(df_ret)

2020-09-16 16:02:17,167 - INFO - start loading target_1m30+


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=300,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=0, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)


2020-09-16 16:03:17,869 - INFO - finish loading target_1m30+
2020-09-16 16:03:18,256 - INFO - start train merge fill zeros attribute:1m30+
2020-09-16 17:47:00,529 - INFO - start loading target_2m30+
2020-09-16 17:48:23,375 - INFO - finish loading target_2m30+
2020-09-16 17:48:23,778 - INFO - start train merge fill zeros attribute:2m30+
2020-09-16 19:02:26,982 - INFO - start loading target_3m30+
2020-09-16 19:03:19,333 - INFO - finish loading target_3m30+
2020-09-16 19:03:19,712 - INFO - start train merge fill zeros attribute:3m30+
2020-09-16 20:55:02,725 - INFO - start loading target_4m30+
2020-09-16 20:58:04,759 - INFO - finish loading target_4m30+
2020-09-16 20:58:05,141 - INFO - start train merge fill zeros attribute:4m30+


,1m30+,new_1m30+,old_1m30+,2m30+,new_2m30+,old_2m30+,3m30+,new_3m30+,old_3m30+,4m30+,new_4m30+,old_4m30+
train,0.999010,0.999395,0.998299,0.998121,0.998862,0.996766,0.998782,0.99914,0.998453,0.998654,0.999141,0.998121
test,0.599606,0.569888,0.629322,0.643056,0.616296,0.649195,0.654374,0.62952,0.655225,0.655701,0.633685,0.654798
